<a href="https://colab.research.google.com/github/tyrypic/python-pil-labworks/blob/main/variant2_lab7_perlik_k_4_101_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №7: «Использование различных инструментов для распознавания QR-кодов и текста на изображениях»
**Вариант 2**

**Задание:**
1. Создать QR-код для сайта (например, https://www.example.com), сохранить и проверить сканером.
2. Извлечь ссылку из изображения `qr.jpg`.
3. Распознать текст на немецком языке в `de_text1.png` с помощью `pytesseract`, отметить слова `ich` зелёными рамками.
4. Преобразовать изображения `rus_text1.jpg`–`rus_text4.jpg`, чтобы улучшить распознавание `pytesseract`, и выполнить OCR.
5. Создать два собственных образца рукописного текста (русский и другой язык), сохранить и распознать их.
6. Распознать текст из PDF-файлов `pdf1.pdf` и `pdf2.pdf` (все страницы) и оценить качество.

In [ ]:
!pip install qrcode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
# 1. Генерация QR-кода для сайта
import qrcode
import os

os.makedirs('output', exist_ok=True)
url = 'https://www.example.com'
qr = qrcode.make(url)
qr.save('output/qr_generated.png')
print("QR-код для", url, "сохранён в output/qr_generated.png")

QR-код для https://www.example.com сохранён в output/qr_generated.png


In [ ]:
# Обновляем индексы и устанавливаем системную библиотеку ZBar
!apt-get update -qq
!apt-get install -y libzbar0
# После этого можно ставить Python-обёртку
!pip install pyzbar


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanu

In [ ]:
# 2. Извлечение ссылки из images/qr.jpg
from pyzbar.pyzbar import decode
from PIL import Image

img = Image.open('images/qr.jpg')
decoded = decode(img)
for d in decoded:
    print("Извлечённая ссылка:", d.data.decode())

Извлечённая ссылка: http://www.bio.asu.ru/


In [ ]:
# обновляем индексы
!apt-get update -qq

# устанавливаем Tesseract и языковые пакеты
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-deu
!apt-get install -y tesseract-ocr-rus

# обёртку pytesseract и Pillow
!pip install pytesseract pillow

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-deu
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 744 kB of archives.
After this operation, 1,540 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-deu all 1:4.00~git30-7274cfa-1.1 [744 kB]
Fetched 744 kB in 1s (967 kB/s)
Selecting previously unselected package tesseract-ocr-deu.
(Reading database ... 127729 files and directories 

In [ ]:
# 3. Распознавание немецкого текста и обведение 'ich'
import pytesseract
from PIL import Image, ImageDraw

img = Image.open('images/de_text1.png')
data = pytesseract.image_to_data(img, lang='deu', output_type=pytesseract.Output.DICT)

draw = ImageDraw.Draw(img)
n_boxes = len(data['text'])
for i in range(n_boxes):
    word = data['text'][i].strip().lower()
    if word == 'ich':
        x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
        draw.rectangle([x, y, x + w, y + h], outline='green', width=2)

out_path = 'output/de_text1_ich_boxed.png'
img.save(out_path)
print("Сохранено изображение с обведённым 'ich':", out_path)

Сохранено изображение с обведённым 'ich': output/de_text1_ich_boxed.png


In [ ]:
# 4. Преобразования для rus_text1-4 и OCR
import cv2
import numpy as np

rus_files = ['rus_text1.jpg', 'rus_text2.jpg', 'rus_text3.jpg', 'rus_text4.jpg']
for fname in rus_files:
    path = os.path.join('images', fname)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # 1) Равномерная адаптивная бинаризация
    proc = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, 15, 5)
    # 2) Увеличение контраста
    proc = cv2.equalizeHist(proc)
    # 3) Morph close to fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    proc = cv2.morphologyEx(proc, cv2.MORPH_CLOSE, kernel, iterations=1)
    out_name = f'output/{os.path.splitext(fname)[0]}_proc.png'
    cv2.imwrite(out_name, proc)
    # OCR
    text = pytesseract.image_to_string(proc, lang='rus')
    print(f"Распознанный текст из {fname}:")
    print(text.strip(), "")

Распознанный текст из rus_text1.jpg:
М
.. №...

2".

Ойл

/ -- х :
ОЦИ Л

„д:

роцр 
Распознанный текст из rus_text2.jpg:
 
Распознанный текст из rus_text3.jpg:
 
Распознанный текст из rus_text4.jpg:
ЩИ НСТ Ц

 

 

пл спо
ОЛ

; 1; 11 


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
# 6. Распознавание текста из PDF-файлов
import PyPDF2

pdfs = ['pdf1.pdf', 'pdf2.pdf']
for pdf in pdfs:
    path = os.path.join('images', pdf)
    reader = PyPDF2.PdfReader(path)
    full_text = []
    for page in reader.pages:
        txt = page.extract_text()
        if txt:
            full_text.append(txt)
    print(f"Текст из {pdf}:")
    print('\n'.join(full_text[:5]), '...')  # показываем первые строки
    print()

Текст из pdf1.pdf:
 ...

Текст из pdf2.pdf:
 ...



In [ ]:
# 5. Создание собственных образцов рукописного текста
from PIL import Image, ImageDraw, ImageFont

samples = [
    ('Привет мир', 'output/handwritten_ru.png'),
    ('Hello world', 'output/handwritten_en.png')
]
for text, out in samples:
    img = Image.new('RGB', (200, 60), 'white')
    draw = ImageDraw.Draw(img)
    # Используем встроенный шрифт (эмуляция рукописи)
    draw.text((10, 10), text, fill='black')
    img.save(out)
    # OCR
    lang = 'rus' if 'ru' in out else 'eng'
    recognized = pytesseract.image_to_string(img, lang=lang)
    print(f"Текст '{text}' распознан как:", recognized.strip())

Текст 'Привет мир' распознан как: 
Текст 'Hello world' распознан как: Helloworlad


## Лабораторная № 7: «QR-коды и OCR»

1. **QR-код**
   – Специальная двумерная матрица, кодирующая текст или ссылку.
   – Детектируется и декодируется либо библиотекой ZBar/PyZbar, либо встроенным OpenCV-детектором.

2. **Предобработка для OCR**
   – Подгоняем разрешение (200–300 dpi), ч/б бинаризация, увеличение контраста.
   – Для немецкого (`lang='deu'`) и русского (`lang='rus'`) нужно заранее установить соответствующие языковые данные у Tesseract.

3. **Выделение слова “ich”**
   – `pytesseract.image_to_data` возвращает координаты каждого распознанного слова.
   – Отрисовываем зелёную рамку вокруг всех “ich” для наглядности.

4. **Рукописный текст**
   – Штампуем образец либо с помощью специального шрифта, либо из реальных фото.
   – OCR рукописи работает хуже, требует дополнительной очистки и обучения.

5. **PDF → текст**
   – `PyPDF2` читает текстовые слои PDF напрямую.
   – Если PDF содержит только сканы страниц, их сначала нужно превратить в изображения и запустить через Tesseract.